# Lista 4 (11 pkt.)

In [1]:
import numpy as np

Ciało Galois (ciało skończone) $GF(p^n)$ ma $p^n$ elementów gdzie $p$ jest liczbą pierwszą a $n$ liczbą naturalną. W ciałach $GF(p)=\{0,1,...,p-1\}$ dodawanie ($\oplus$) i mnożenie ($\odot$) są zdefiniowane jako zwykłe dodawanie ($+$) i mnożenie ($\cdot$) liczb całkowitych modulo $p$.

$$a,b\in GF(p)$$

$$a\oplus b=a+b\mod p$$

$$a\odot b=a \cdot b\mod p$$

W ciałach $GF(p^n)$ elementy ciała możemy traktować jako wielomiany stopnia $n-1$ o współczynnikach z ciała $GF(p)$:

$$a\in GF(p^n)$$

$$a=c_{n-1}x^{n-1}+c_{n-2}x^{n-2}+...+c_1x+c_0$$

$$c_k\in GF(p)$$

Spójrzmy na przypadek, który nas najbardziej interesuje czyli ciala $GF(2^n)$.

Ciało $GF(2)=\{0,1\}$ jest ciałem dwu elemntowym, z działaniami

$0\oplus0=0$, $0\oplus1=1$, $1\oplus1=0$,

$0\odot0=0$, $0\odot1=0$, $1\odot1=1$.

Zatem cialo $GF(2^n)$ zawiera wielomiany o wspólczynnikach $0$ i $1$, np. ciało $GF(2^3)$ zawiera 8 elementów:

$1$, $x$, $x+1$, $x^2$, $x^2+1$, $x^2+x$, $x^2+x+1$,

elementy te możemy reprezentować jako ciąg bitów, określający współczynnki w wielomianie, tzn.

$1=001$, $x=010$, $x+1=011$, $x^2=100$, $x^2+1=101$, $x^2+x=110$, $x^2+x+1=111$.

W poniższych zadaniach będzie trzeba zaimplementować działania w ciele $GF(2^n)$.

## Zadanie 1 (1 pkt.)

Dodawanie w ciele $GF(p^n)$ jest zdefiniowane jako zwykłe dodawanie wielomianów, z tym że wspóczynniki dodają się zgodnie z regułami z ciała $GF(p)$. Zatem w przypadku ciała $GF(2^n)$ dodawnie wyglada na przykład tak:

chcmemy dodać do siebie dwa elementy ciała $GF(2^3)$: $x^2+x$ oraz $x+1$:

$(x^2+x)\oplus (x+1)=(1\cdot x^2+1\cdot x+0\cdot 1)\oplus (0\cdot x^2+1\cdot x+1\cdot 1)=(1\oplus0)\cdot x^2+(1\oplus 1)\cdot x+(0\oplus 1)\cdot 1=1\cdot x^2+0\cdot x+1\cdot 1=x^2+1$

to samo dodawanie możemy zapisać reprezentując wielomiany jako ciągi btów:
$110\oplus011=101$
widzimy więc, że dodawnie wielomianów jest zwykłą alternatywą wykluczająca dwóch ciagów bitów.

Zauważmy też, że w wyniku dodawnia zawsze dostajemy wielomian o stopniu nie wyższym niż składniki działania zatem nie wychodzimy poza ciało, czyli nie musimy wykonywac żadnej operacji modulo na wielomianach.

Zaimplementuj funkcję $\textit{add_GF(p,q)}$, przyjmującą dwa stringi będące ciagami bitów reprezentujące dwa 
wielomiany i zwracjącą string będący ciagiem bitów reprezentujący wielomian $p\oplus q$.

In [2]:
def add_GF(p, q):
    longer = max(len(p), len(q))
    p = p.zfill(longer)
    q = q.zfill(longer)
    result = ''
    for i in range(longer):
        result += str(int(p[i]) ^ int(q[i]))
    result = result.lstrip('0')
    return result

In [3]:
print(add_GF('110','11')=='101')
print(add_GF('110','101')=='11')

True
True


Poza tym zaimplementuj funkcję zamieniającą string z bitami na string zawierający zapis szesnastkowy (uzupełniający ewentualnie zerami z przodu aby uzyskać zadaną liczbę cyfr $\textit{pad}$), przyda nam się to później.

In [4]:
def bin2hex(bin_str,pad):
    number = hex(int(bin_str,2)).replace('0x','')
    pad = int(pad)
    while len(number)<pad:
        number = '0' + number
        
    #print(number)
    return number

In [5]:
print(bin2hex('1101',2)=='0d')
print(bin2hex('11010011',2)=='d3')
print(bin2hex('1111111',3)=='07f')
print(bin2hex('1',1)=='1')

True
True
True
True


## Zadanie 2 (2 pkt.)

Teraz zajmiemy się mnożeniem, które jest już bardziej skomplikowane. Na początek mnożymy jak w przypadku zwykłych wielomianów, z tym, że współczynniki mnożą się zgodnie z regułami $GF(2)$. Jak łatwo jednak zauważyć w wyniku mnożenia możemy dostać wielomian o stopniu wyższym niż mnożone wielomiany zatem nie należący do naszego ciał, aby wynik zawsze należał do ciała musimy dodatkowo wykonać dzielenie wielomianu modulo pewien nieredukowalny wielomian, który musi być podany jako elelment definicji, gdyż takich możliwych wielomianów może być wiele.

Zanim jednak będziemy cokolwiek dzielić musimy zaimplementować zwykłe mnożenie wielomianów.

Zaimplementuj funkcję $\textit{multiply(p,q)}$, przyjmującą dwa stringi będące ciagami bitów reprezentujące dwa 
wielomiany i zwracjącą string będący ciagiem bitów reprezentujących wielomian $pq$. Nie jest to mnożenie w ramach ciała Galois bo jeszcze nie uwzględniamy operacji modulo, ale dodawnie, które się pojawia przy mnożeniu tych wielomianów jest dodawaniem w ciele Galois ($\textit{add_GF()}$), tzn. na przykład:

$(x^2+x)(x+1)=x^3+x$ bo $x^2\oplus x^2=0$

jak łatwo zauważyć, $x^3+x\notin GF(2^3)$

In [6]:
def multiply(p, q):
    result = ''
    occurence_dict = {}

    longer = max(len(p), len(q))
    p = p.zfill(longer)
    q = q.zfill(longer)
    
    for i in range(len(p)):
        for j in range(len(q)):
            if j + i in occurence_dict.keys():
                occurence_dict[i + j].append(str(int(p[i]) * int(q[j])))
            else:
                occurence_dict[i + j] = [str(int(p[i]) * int(q[j]))]
    for occurence_value in occurence_dict.values():
        occurence = occurence_value[0]
        for i in range(1, len(occurence_value)):
            occurence = add_GF(occurence, occurence_value[i])
            if (occurence == ''):
                occurence = '0'
        result += occurence
    result = result.lstrip('0')
    return result

In [7]:
print(multiply('10','1')=='10')
print(multiply('10','10')=='100')
print(multiply('110','11')=='1010')

True
True
True


## Zadanie 3 (2 pkt.)

Zaimplementuj funkcję $\textit{divide(p,q)}$, która dla wielomianów $p$ i $q$ zwraca wynik dzielenie $p$ przez $q$ wraz z resztą. Jak dzielić wielomiany z resztą przypomnij sobie ze szkoły średniej, tu będzie tak samo tylko trzeba pamietać, że współczynniki w naszych wielomianach należą do ciała $GF(2)$. Może być wygodne również zauważenie, że w przypadku $GF(2)$ dodawanie jest tożsame z odejmownaniem, tj. dla $a,b\in GF(2)$ mamy $a+b=a-b$ zatem w schemacie pisemnego mnożenia wielomianów możemy dodawać zamiast odejmować.

In [8]:
def divide(p, q):
    result = '0'
    q = q.lstrip('0')
    if q == '1':
        return p, '0'
    while len(p) >= len(q):
        polynomial_degree = len(p) - len(q)
        r = '1' + polynomial_degree * '0'
        result = add_GF(result, r)
        p = add_GF(p, multiply(q, r)) 
        if (p == ''):
            p = '0'
    return result, p

In [9]:
print(divide('1011','11')==('110','1'))
print(divide('1010','110')==('11','0'))
print(divide('1111','10')==('111','1'))

True
True
True


## Zadanie 4 (1 pkt.)

Teraz możemy przejść do mnożenia w ciele $GF(2^n)$. Jest to zwykłe mnożenie wielomianów (funkcja $\textit{multiply()}$) ale dodatkowo musimy wykonać dzielenie z resztą (funkcja $\textit{divide()}$) przez pewnie nieredukowalny wielomian, i ostatecznym wynikiem jest właśnie reszta z tego dzielenia. Wielomian ten można wybrać na wiele sposobów.

Skupmy się więc od teraz na przypadku, który nas najbardziej interesuje czyli ciele Galois używanym w przypadku szyfrowania AES, któym będziemy się zajmować. Ciałem tym jest $GF(2^8)$ z wielomianem nieredukowalnym $x^8+x^4+x^3+x+1$.

Zaimplementuj funkcję $\textit{multiply_GF(p,q)}$, która dla wielomianów $p$, $q$ zwraca wynika mnożenia $p\odot q$ w ciele $GF(2^8)$ modulo wielomian $x^8+x^4+x^3+x+1$.

In [10]:
def multiply_GF(p, q):
    polynomial = '100011011'
    result = multiply(p, q)  

    if (len(result) > len(polynomial) - 1):
        result = divide(result, polynomial)[1]
    return result

In [11]:
print(multiply_GF('1101','1')=='1101')
print(multiply_GF('1101','10')=='11010')
print(multiply_GF('10000000','10')=='11011')
print(multiply_GF('11100101','1010')=='10111000')

True
True
True
True


## Zadanie 5 (2 pkt.)

Potrafimy mnożyć już wielomiany w ciele $GF(2^8)$, chcielibyśmy też umieć znajdować do danego elementu $p$ element odwrotny $q$, to jest taki, że $p\odot q=1$. Aby to zrobić będziemy musieli skorzystać z Rozszerzonego Algorytmu Euklidesa (EEA), algorytm ten dla danych liczb całkowitych $a$ i $b$ znajduje takie liczby całkowite $c$ i $d$, że:

$$ac+bd=NWD(a,b)$$
gdzie $NWD$ to największy wspólny dzielnik.

Ten sam algorytm możemy użyć w odniesieniu do elementów ciała $GF(2^8)$, wystarczy wszystkie operacje dodawania, mnożenia i dzielenia z resztą używane w tym algorytmie zastąpić operacjami $\textit{add_GF()}$, $\textit{multiply_GF()}$, $\textit{divide()}$.

Zaimplementuj funkcję $\textit{EEA_GF(p,q)}$, która dla wielomianów $p$, $q$ zwraca wielomiany $s$ i $t$ takie, że $s\odot p\oplus t\odot q=NWD(p,q)$.

In [12]:
# NWD(p,q) =s * p +  t * q 

def EEA_GF(r0, r1):
    s = '0'
    t = '1'
    s_previous = '1'
    t_previous = '0'
    
    while (r0 != '0'):
        division = divide(r1, r0)
        s_next = add_GF(s, multiply(s_previous, division[0]))
        t_next = add_GF(t, multiply(t_previous, division[0]))
        r1, r0 = r0, division[1]
        s, t = s_previous, t_previous
        s_previous, t_previous = s_next, t_next
    return s, t

In [13]:
print(EEA_GF('11010101','10010111')==('11001', '10100'))
print(EEA_GF('11110000','11001011')==('1000000', '1010111'))

True
True


## Zadanie 6 (1 pkt.)

Korzytsając z Rozszerzonego Algorytmu Euklidesa (EEA) możemy znajdować wielomian odwrotny do danego. Pozostańmy przy naszym przypadku ciała $GF(2^8)$ z wielomianem nieredukowalnym $m=x^8+x^4+x^3+x+1$. Weźmy dowolny wielomian $p\in GF(2^8)$. Korzystając z EEA możemy znaleźć takie $s$ i $t$, że $$s\odot p\oplus t\odot m=NWD(p,m)$$

ponieważ $m$ jest nieredukowalny to $NWD(p,m)=1$ a zatem

$$s\odot p\oplus t\odot m=1$$

poza tym zauważmy, że $t\odot m=0$, bo przy wykonywaniu działania $\odot$ dzielimy modulo własnie przez $m$ więc reszta z tego dzielenia jest zerem. Zatem:

$$s\odot p=1$$

czyli znaleźliśmy element odwrotny do $p$ jest to $s$, czyli jeden z wielomianów, które zwraca algorytm $EEA$.

Napisz funkcję zwracjącą odwrotność podanego wielomianu, korzystającą z funkcji $\textit{EEA_GF()}$

In [14]:
def inverse_GF(p):
    if (p == '0' * 8):
        return p
    polynom = '100011011'

    return EEA_GF(p, polynom)[0]

In [15]:
print(inverse_GF('11001010')=='1010011')
print(inverse_GF('1100101')=='10100110')

True
True


## Zadanie 7 (2 pkt.)

Na poprzedniej liście dotyczącej DES-a używaliśmy S-Boxa. W przypadku AES-a również jest używany odpowiedni S-Box, zamienia on ciąg 8 bitów na inny ciąg 8 bitów. Poniżej widzimy gotową postać tego S-Boxa. Mając dane 8 bitów (np. '01010110') dzielimy je na dwie grupy ('0101' i '0110'), zamieniając je na liczbę dziesiętną otrzymujemy numer wiersza oraz kolumny (5 i 6), z których odczytujemy wynik ('b1'), który jest zapisany w postaci szesnastkowej, zamieniamy go więc na binarną ('10110001') i to jest ciąg bitów, którym zastępujemy wejściowy.

W przypadku DES-a S-Box był dany jako element specyfikacji, tutaj natomiast S-Box wynika z teorii ciał Galois i możemy go wyprowadzić, tego dotyczy właśnie to zadanie. Operacje, które stoją za działaniem S-Boxa są następujące. Bierzemy wejściowy ciąg 8 bitów $p$ następnie obliczamy jego odwrotność w ciele Galois (funkcja $\textit{inverse_GF()}$) to co otrzymujemy oznaczmy $p^{-1}$, następnie dokonujemy transformacji afinicznej za pomocą macierzy $A$ oraz wektora $v$

$$q=Ap^{-1}+v$$

gdzie mamy iloczyn macierzy i wektora ($p$ traktujemy jako wektor bitów, bez zmiany kolejności, tj. $p=[p_7, p_6, ..., p_0]$ ) oraz sumę wektorów, przy czym operacje na bitach wykonujemy w ramach ciała $GF(2)$. Otrzymany ciąg bitów $q$ jest naszym wynikiem, ten ciąg właśnie znajduje się na odpowiendnim dla $p$ miejscu S-Boxa.

Uwaga: wektor $p=(0,0,0,0,0,0,0,0)$ nie ma odwrotności w ciele $GF(2^8)$ bo jest wektorem zerowym a ciało tworzy grupę ze względu na mnożenie dla wszystkich elementów poza zerowym (czyli tym, który jest elementem neutralnym w działaniu dodawania) w ramach algorytmu AES przyjmujemy, że $p^{-1}$ jest też wektorem zerowym ($p^{-1}=(0,0,0,0,0,0,0,0)$).

Napisz funkcję $\textit{substitute(p)}$, ktore przyjmuje ciąg bitów i zwraca ciąg (reprezentowany przez dwie cyfry szensastkowe) wynikający z powyższej transformacji. A następnie wygeneruj za jej pomocą S-Box, sprawdź czy wyszedł taki jak poniżej, użyj funkcji $\textit{bin2hex()}$ aby zamienić ciągi bitów na zapis szesnastkowy.

In [16]:
SBox=[['63', '7c', '77', '7b', 'f2', '6b', '6f', 'c5', '30', '01', '67', '2b', 'fe', 'd7', 'ab', '76'],
      ['ca', '82', 'c9', '7d', 'fa', '59', '47', 'f0', 'ad', 'd4', 'a2', 'af', '9c', 'a4', '72', 'c0'],
      ['b7', 'fd', '93', '26', '36', '3f', 'f7', 'cc', '34', 'a5', 'e5', 'f1', '71', 'd8', '31', '15'],
      ['04', 'c7', '23', 'c3', '18', '96', '05', '9a', '07', '12', '80', 'e2', 'eb', '27', 'b2', '75'],
      ['09', '83', '2c', '1a', '1b', '6e', '5a', 'a0', '52', '3b', 'd6', 'b3', '29', 'e3', '2f', '84'],
      ['53', 'd1', '00', 'ed', '20', 'fc', 'b1', '5b', '6a', 'cb', 'be', '39', '4a', '4c', '58', 'cf'],
      ['d0', 'ef', 'aa', 'fb', '43', '4d', '33', '85', '45', 'f9', '02', '7f', '50', '3c', '9f', 'a8'],
      ['51', 'a3', '40', '8f', '92', '9d', '38', 'f5', 'bc', 'b6', 'da', '21', '10', 'ff', 'f3', 'd2'],
      ['cd', '0c', '13', 'ec', '5f', '97', '44', '17', 'c4', 'a7', '7e', '3d', '64', '5d', '19', '73'],
      ['60', '81', '4f', 'dc', '22', '2a', '90', '88', '46', 'ee', 'b8', '14', 'de', '5e', '0b', 'db'],
      ['e0', '32', '3a', '0a', '49', '06', '24', '5c', 'c2', 'd3', 'ac', '62', '91', '95', 'e4', '79'],
      ['e7', 'c8', '37', '6d', '8d', 'd5', '4e', 'a9', '6c', '56', 'f4', 'ea', '65', '7a', 'ae', '08'],
      ['ba', '78', '25', '2e', '1c', 'a6', 'b4', 'c6', 'e8', 'dd', '74', '1f', '4b', 'bd', '8b', '8a'],
      ['70', '3e', 'b5', '66', '48', '03', 'f6', '0e', '61', '35', '57', 'b9', '86', 'c1', '1d', '9e'],
      ['e1', 'f8', '98', '11', '69', 'd9', '8e', '94', '9b', '1e', '87', 'e9', 'ce', '55', '28', 'df'],
      ['8c', 'a1', '89', '0d', 'bf', 'e6', '42', '68', '41', '99', '2d', '0f', 'b0', '54', 'bb', '16']]

In [17]:
A=[[1, 1, 1, 1, 1, 0, 0, 0],
   [0, 1, 1, 1, 1, 1, 0, 0],
   [0, 0, 1, 1, 1, 1, 1, 0],
   [0, 0, 0, 1, 1, 1, 1, 1],
   [1, 0, 0, 0, 1, 1, 1, 1],
   [1, 1, 0, 0, 0, 1, 1, 1],
   [1, 1, 1, 0, 0, 0, 1, 1],
   [1, 1, 1, 1, 0, 0, 0, 1]]

v=[0, 1, 1, 0, 0, 0, 1, 1]

In [18]:
def substitute(p):
    a_plus_vector = ''
    ap = []
    inversed_p = inverse_GF(p)
    inversed_p = inversed_p.zfill(8)

    for i in range(len(A)):
        multiplied = 0
        for j in range(len(A[0])):
            multiplied += A[i][j] * int(inversed_p[j])
        ap.append(multiplied)
    for i in range(len(ap)):
        a_plus_vector += str((ap[i] + v[i]) % 2)
    to_x = bin2hex(a_plus_vector, 2)
    return to_x

In [19]:
print(substitute('11010101')=='03')
print(substitute('01010001')=='d1')
print(substitute('11011101')=='c1')
print(substitute('11101100')=='ce')

True
True
True
True


In [20]:
s_box = []

for i in range(len(SBox)):
    one_row = []
    row_index = format(i, 'b').zfill(4)
    
    for j in range(len(SBox[0])):
        col_index = format(j, 'b').zfill(4)
        p = row_index + col_index
        one_row.append(substitute(p))
    s_box.append(one_row)

In [21]:
print(s_box==SBox)

True
